### **ETL - steam_games**
**En este notebook realizaremos la extracción, transformación y carga (ETL) de `steam_games`**
**Al Finalizar se exportara la información extraida para un posterior análisis EDA**

** Los data Sets estan disponibles en  link **

1.  leemos los archivos que contiene el data set de steam_games

In [1]:

# Importando librerías
import json                     # Módulo de codificador y decodificador JSON
import pandas as pd             # Librería para manipular datasets
import pyarrow as pa            # Útil para realizar operaciones de lectura y escritura de datos
import pyarrow.parquet as pq    # Útil para leer y escribir datos en formato Parquet de manera eficiente 
import os                       # creación de directorios y comprobación de existencia, api kernel del SO
import gzip                     # Abrir directorios comprimidos en zip
import re                       # Para Trabajar con expresioner regulares

2. El Formato que tiene el archivo Json viene mal estructurado, entonces recorremos cada linea decodificando en UTF 8 para despues agregar a un array vacio, cada un haciendo un parsear a Json con el metodo loads. Al finalizar la lectura de cada linea del archivo armamos el dataframe con el array.

In [2]:
# Cargo el Dataframe de Games
#GamesDF

data_list = []

steam_games_path = 'C:\\Users\\ahurt\\OneDrive\\Escritorio\\Introduction to IA\\Henry\\GameRecommenderX\\DataSets\\steam_games.json.gz'
# Abre el archivo comprimido .json.gz
with gzip.open(steam_games_path, 'rb') as f:
    # Lee cada línea del archivo descomprimiendo y decodificando como UTF-8
    for line in f:
        json_content = line.decode('utf-8')
        # Carga cada línea como un objeto JSON y agrega a la lista
        data_list.append(json.loads(json_content))

# Convierte la lista de datos JSON en un DataFrame de Pandas
GamesDF = pd.DataFrame(data_list)

3. Se hace un analisis de la estructura de la informacion

In [3]:
GamesDF

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [4]:
GamesDF.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [5]:
GamesDF.describe()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
count,24083,28852,32133,30085,32135,30068,31972,32133,31465,30758.00,32135,32133,28836
unique,8239,883,32094,30054,32135,3582,15395,32132,4649,162.00,2,32132,10992
top,Ubisoft,[Action],Soundtrack,Soundtrack,http://store.steampowered.com/app/761140/Lost_...,2012-10-16,"[Casual, Simulation]",http://steamcommunity.com/app/612880/reviews/?...,[Single-player],4.99,False,612880,Ubisoft - San Francisco
freq,385,1880,3,3,1,100,1292,2,2794,4278.00,30188,2,1259


# Tratamiento de los datos nulos

In [6]:
# Identificar cuantos de los datos son nulos
Total_nulos= GamesDF.isnull().sum()
Total_nulos

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [7]:
# Borrare los nulos de cada columna

GamesDF = GamesDF.dropna(subset=['genres'])
GamesDF = GamesDF.dropna(subset=['app_name'])
GamesDF = GamesDF.dropna(subset=['release_date'])
GamesDF = GamesDF.dropna(subset=['price'])
GamesDF = GamesDF.dropna(subset=['id'])
GamesDF = GamesDF.dropna(subset=['developer'])

In [8]:
# Re asignamos los indices
GamesDF= GamesDF.reset_index(drop=True)

In [9]:
GamesDF["app_name"]

0             Lost Summoner Kitty
1                       Ironbound
2         Real Pool 3D - Poolians
3                         弹炸人2222
4           Battle Royale Trainer
                   ...           
27457                Kebab it Up!
27458              Colony On Mars
27459    LOGistICAL: South Africa
27460               Russian Roads
27461         EXIT 2 - Directions
Name: app_name, Length: 27462, dtype: object

4. Los nombres de los desarrolladores en la columna app_name del DataFrame games son estandarizados mediante la eliminación de caracteres no alfabéticos, la conversión de todas las letras a minúsculas y la supresión de cualquier espacio extra.

In [10]:

def normalize_str(name):
    if isinstance(name, str):  
        cleaned_name = re.sub(r'[^a-zA-Z\s]', '', name)
        normalized_name = re.sub(r'\s+', ' ', cleaned_name.lower().strip())
        return normalized_name
    else:
        return name



Aplico la funcion de normalizacion a todas las columnas que contengan Str

In [11]:
GamesDF

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27457,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,False,745400,Bidoniera Games
27458,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
27459,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
27460,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich


In [12]:
GamesDF['app_name'] = GamesDF['app_name'].apply(normalize_str)
# GamesDF['developer'] = GamesDF['developer'].apply(normalize_str)
# GamesDF['title'] = GamesDF['title'].apply(normalize_str)

Normalizamos la columna Price, Como se puede observar esta compuesta por una columna que tiene strings y floats

In [13]:
GamesDF['price']

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                3.99
             ...     
27457            1.99
27458            1.99
27459            4.99
27460            1.99
27461            4.99
Name: price, Length: 27462, dtype: object

In [14]:

# aqui se aplica una función lambda a cada elemento de la columna 'price' del DataFrame GamesDF.
# Si el elemento es una cadena de texto (str), se reemplaza por 0; de lo contrario, se deja sin cambios.
GamesDF['price'] = GamesDF['price'].apply(lambda x: 0 if isinstance(x, str) else x)



In [15]:
GamesDF['price'] 

0        4.99
1        0.00
2        0.00
3        0.99
4        3.99
         ... 
27457    1.99
27458    1.99
27459    4.99
27460    1.99
27461    4.99
Name: price, Length: 27462, dtype: float64

De esta manera ya tenemos los datos normalizados y listos para ser analisados a mayor profundidad. para no tener que repetir todo el proceso de computo hasta este punto, se hace una copia en formato parquet

In [16]:
ruta_carpeta = "Data_Extracted"

if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

# Asigno la ruta donde quiero guardar el parquet con el nombre que va tener
ruta_guardar_parquet = "Data_Extracted/DataFrame_games.parquet"

# Transformo a una tabla el DataFrame y luego guardo como archivo Parquet
table = pa.Table.from_pandas(GamesDF)
pq.write_table(table, ruta_guardar_parquet)